In [1]:
import numpy as np
import pandas as pd 

In [2]:
# dataset dei sintomi, come base per analisi
df_sym = pd.read_csv('symptoms_gen.csv')
df_sym.head()

,Unnamed: 0,user_id,time,sore.throat.cough,runnynose.congestion.sneezing,fever,nausea.vomiting.diarrhea,sad.depressed,open.stressed
0,0,2,2009-01-09 19:38:00,0,0,0,0,1,1
1,1,2,2009-01-10 13:36:00,0,0,0,0,1,1
2,2,2,2009-01-11 12:58:00,0,0,0,0,0,0
3,3,2,2009-01-12 09:43:00,0,0,0,1,0,0
4,4,2,2009-01-13 12:18:00,0,0,0,0,0,0


In [3]:
df_sym.drop(columns=['Unnamed: 0'],inplace=True)
df_sym.rename(columns={'sore.throat.cough':'cough', 'runnynose.congestion.sneezing':'congestion',
                     'nausea.vomiting.diarrhea':'nausea','sad.depressed':'depressed', 'open.stressed':'stress'}, inplace=True)
df_sym.head()

,user_id,time,cough,congestion,fever,nausea,depressed,stress
0,2,2009-01-09 19:38:00,0,0,0,0,1,1
1,2,2009-01-10 13:36:00,0,0,0,0,1,1
2,2,2009-01-11 12:58:00,0,0,0,0,0,0
3,2,2009-01-12 09:43:00,0,0,0,1,0,0
4,2,2009-01-13 12:18:00,0,0,0,0,0,0


In [4]:
#dataset proximity, per estrarre features

df_pro = pd.read_csv('proximity_2009_gen.csv')
df_pro.drop(columns=['Unnamed: 0'],inplace=True)
df_pro.rename(columns={'remote.user.id.if.known':'remote.user'}, inplace=True)
df_pro.head()

,user.id,remote.user,time,prob2
0,46,20,2009-01-01 00:00:06,0.628
1,46,30,2009-01-01 01:00:24,0.371
2,61,61,2009-01-01 01:36:06,NaN
3,18,12,2009-01-01 02:29:37,0.197
4,46,30,2009-01-01 03:36:35,0.371


In [5]:
#per time tengo solo le date, senza orari
df_pro['time']= pd.to_datetime(df_pro['time']).dt.date
df_pro.head()

,user.id,remote.user,time,prob2
0,46,20,2009-01-01,0.628
1,46,30,2009-01-01,0.371
2,61,61,2009-01-01,NaN
3,18,12,2009-01-01,0.197
4,46,30,2009-01-01,0.371


In [6]:
# anche per i sintomi tengo la data senza orario
df_sym['time']= pd.to_datetime(df_sym['time']).dt.date

In [7]:
# raddoppio le colonne cosi ogni individuo è visto sia come A (su cui poi si fa la logistica) sia come B

range_df_pro = df_pro.index
df_pro = df_pro.loc[np.repeat(df_pro.index.values, 2)]
df_pro.index=range(len(df_pro))

cond = df_pro.index % 2 != 0
df_pro.loc[cond, ['user.id', 'remote.user']] = df_pro.loc[cond, ['remote.user', 'user.id']].values

In [8]:
#calcolo numero incontri giornalieri
df_pro_daily = df_pro.groupby(['user.id','time'], as_index=False).agg({'remote.user':'count'})
df_pro_daily.rename(columns={'remote.user': "n_contacts_24"}, inplace=True)
df_pro_daily.head()

,user.id,time,n_contacts_24
0,1,2009-01-01,4
1,1,2009-01-02,28
2,1,2009-01-03,16
3,1,2009-01-04,1
4,1,2009-01-05,3


In [9]:
#calcolo incontri ogni 48 ore

from datetime import timedelta

df_pro_daily['n_contacts_48']=''     #nuova colonna 
df_pro_daily.loc[0, 'n_contacts_48'] = df_pro_daily.loc[0, 'n_contacts_24'] #compilo la prima riga
n_isolated = 1
for i in range(1, len(df_pro_daily)):
    
    if (df_pro_daily.loc[i, 'user.id'] == df_pro_daily.loc[i-1, 'user.id'] and 
        df_pro_daily.loc[i, 'time'] - timedelta(days=1) == df_pro_daily.loc[i-1, 'time']):
        
        df_pro_daily.loc[i, 'n_contacts_48'] = df_pro_daily.loc[i-1, 'n_contacts_24'] + df_pro_daily.loc[i, 'n_contacts_24']
        
    else:
        df_pro_daily.loc[i, 'n_contacts_48'] = df_pro_daily.loc[i, 'n_contacts_24']
        n_isolated +=1

In [10]:
#calcolo incontri ogni 72 ore

from datetime import timedelta

df_pro_daily['n_contacts_72']=''     #nuova colonna 
df_pro_daily.loc[0, 'n_contacts_72'] = df_pro_daily.loc[0, 'n_contacts_48'] #compilo la prima riga
df_pro_daily.loc[1, 'n_contacts_72'] = df_pro_daily.loc[1, 'n_contacts_48'] #compilo la seconda riga

for i in range(2, len(df_pro_daily)):
    
    df_pro_daily.loc[i, 'n_contacts_72'] = df_pro_daily.loc[i, 'n_contacts_24']
    
    if (df_pro_daily.loc[i, 'user.id'] == df_pro_daily.loc[i-1, 'user.id']  and 
        df_pro_daily.loc[i, 'time'] - timedelta(days=1) == df_pro_daily.loc[i-1, 'time']):
        
        df_pro_daily.loc[i, 'n_contacts_72'] = df_pro_daily.loc[i, 'n_contacts_72'] + df_pro_daily.loc[i-1, 'n_contacts_24']
        
            
    if (df_pro_daily.loc[i, 'user.id'] == df_pro_daily.loc[i-2, 'user.id'] and 
        df_pro_daily.loc[i, 'time'] - timedelta(days=2) == df_pro_daily.loc[i-2, 'time']):
        
        df_pro_daily.loc[i, 'n_contacts_72'] = df_pro_daily.loc[i, 'n_contacts_72'] + df_pro_daily.loc[i-2, 'n_contacts_24']
        
    
    if (df_pro_daily.loc[i, 'user.id'] == df_pro_daily.loc[i-1, 'user.id'] and 
        df_pro_daily.loc[i, 'time'] - timedelta(days=2) == df_pro_daily.loc[i-1, 'time']): #caso in cui c'è un buco di un giorno
        
        df_pro_daily.loc[i, 'n_contacts_72'] = df_pro_daily.loc[i, 'n_contacts_72'] + df_pro_daily.loc[i-1, 'n_contacts_24']
        

In [11]:
df_pro_daily.head()
df_pro_daily[df_pro_daily['user.id']==10]

,user.id,time,n_contacts_24,n_contacts_48,n_contacts_72
654,10,2009-01-04,15,15,15
655,10,2009-01-07,1,1,1
656,10,2009-01-10,3,3,3
657,10,2009-01-26,6,6,6
658,10,2009-01-27,1,7,7
659,10,2009-01-29,4,4,5
660,10,2009-02-04,5,5,5
661,10,2009-02-12,329,329,329
662,10,2009-02-18,2203,2203,2203
663,10,2009-02-20,3,3,2206


In [12]:
#controllo quante volte manca il giorno precedente
print('isolated: ',n_isolated,' | total: ', len(df_pro_daily))

isolated:  487  | total:  5829


In [13]:
# per fare il merge considero i dati di proximity di due giorni prima. aggiungo la data che mi interessa del dataset symptoms
#df_sym['time_contagion'] = df_sym['time'] - timedelta(days=2)

In [14]:
#merge
#df_pro_daily.rename(columns={'time': "time_proximity"}, inplace=True)
df = df_sym.merge(df_pro_daily, how='left', left_on=['user_id', 'time'], right_on=['user.id', 'time'])
df.drop(columns=['user.id'],inplace=True)
df.head(10)

,user_id,time,cough,congestion,fever,nausea,depressed,stress,n_contacts_24,n_contacts_48,n_contacts_72
0,2,2009-01-09,0,0,0,0,1,1,68.0,110.0,219.0
1,2,2009-01-10,0,0,0,0,1,1,107.0,175.0,217.0
2,2,2009-01-11,0,0,0,0,0,0,95.0,202.0,270.0
3,2,2009-01-12,0,0,0,1,0,0,185.0,280.0,387.0
4,2,2009-01-13,0,0,0,0,0,0,22.0,207.0,302.0
5,2,2009-01-14,0,0,0,0,1,0,57.0,79.0,264.0
6,2,2009-01-15,0,0,0,0,0,0,108.0,165.0,187.0
7,2,2009-01-16,0,0,0,0,1,0,18.0,126.0,183.0
8,2,2009-01-16,0,0,0,0,1,0,18.0,126.0,183.0
9,2,2009-01-17,0,0,0,0,0,0,3.0,21.0,129.0


In [15]:
#controllo quante righe hanno sintomi ma non numero di contatti
print('na: ',len(df[df.isna().any(axis=1)]),' | total: ', len(df))
#df.isna().sum() 

na:  592  | total:  3876


In [16]:
# dataset delle relazioni
df_rel = pd.read_csv('relation_2009.csv')
df_rel.drop(columns=['Unnamed: 0'],inplace=True)
df_rel.head()

,id.A,id.B,relationship,survey.date
0,47,2,CloseFriend,2009-04-17
1,23,2,CloseFriend,2009-04-17
2,57,2,CloseFriend,2009-04-17
3,28,2,CloseFriend,2009-04-17
4,55,2,CloseFriend,2009-04-17


In [17]:
# merge di proximity e relationships
df_pro_rel = df_pro.merge(df_rel, how='left', left_on=['user.id', 'remote.user'], right_on=['id.A', 'id.B'])

df_pro_rel.drop(columns=['id.A','id.B','survey.date'],inplace=True)

df_pro_rel.head()

,user.id,remote.user,time,prob2,relationship
0,46,20,2009-01-01,0.628,CloseFriend
1,46,20,2009-01-01,0.628,PoliticalDiscussant
2,46,20,2009-01-01,0.628,FacebookAllTaggedPhotos
3,46,20,2009-01-01,0.628,BlogLivejournalTwitter
4,20,46,2009-01-01,0.628,FacebookAllTaggedPhotos


In [18]:
#ci sono contatti tra persone che non hanno specificato la relazione. rimpiazzio i NaN con 'Unknown'
df_pro_rel['relationship'].fillna('Unknown', inplace=True)

In [19]:
#numero di incontri giornalieri, diviso per tipo di relazione
df_pro_rel_daily = (df_pro_rel.groupby(['user.id', 'time','relationship'])
         .size()
         .unstack(fill_value=0)
         .reset_index())

df_pro_rel_daily.columns.name = None

df_pro_rel_daily.head()

,user.id,time,BlogLivejournalTwitter,CloseFriend,FacebookAllTaggedPhotos,PoliticalDiscussant,SocializeTwicePerWeek,Unknown
0,1,2009-01-01,0,0,0,0,0,4
1,1,2009-01-02,0,0,0,0,0,28
2,1,2009-01-03,0,0,0,0,0,16
3,1,2009-01-04,0,0,0,0,0,1
4,1,2009-01-05,0,0,0,0,0,3


In [20]:
# funzione per calcolare le 48 ore e le 72 ore per i diversi tipi di contatto

def proximity_48_72(df, col):
    
    #calcolo incontri ogni 48 ore
    
    df[(col+'_48')]=''     #nuova colonna 
    df.loc[0, (col+'_48')] = df.loc[0, col] #compilo la prima riga
    for i in range(1, len(df)):
    
        if (df.loc[i, 'user.id'] == df.loc[i-1, 'user.id'] and 
            df.loc[i, 'time'] - timedelta(days=1) == df.loc[i-1, 'time']):
        
            df.loc[i, (col+'_48')] = df.loc[i-1, col] + df.loc[i, col]
        
        else:
            df.loc[i, (col+'_48')] = df.loc[i, col]
            
            
    #calcolo incontri ogni 72 ore

    df[(col+'_72')]=''     #nuova colonna 
    df.loc[0, (col+'_72')] = df.loc[0, (col+'_48')] #compilo la prima riga
    df.loc[1, (col+'_72')] = df.loc[1, (col+'_48')] #compilo la seconda riga

    for i in range(2, len(df)):
    
        df.loc[i, (col+'_72')] = df.loc[i, col]
    
        if (df.loc[i, 'user.id'] == df.loc[i-1, 'user.id']  and 
            df.loc[i, 'time'] - timedelta(days=1) == df.loc[i-1, 'time']):
        
            df.loc[i, (col+'_72')] = df.loc[i, (col+'_72')] + df.loc[i-1, col]
        
            
        if (df.loc[i, 'user.id'] == df.loc[i-2, 'user.id'] and 
            df.loc[i, 'time'] - timedelta(days=2) == df.loc[i-2, 'time']):
        
            df.loc[i, (col+'_72')] = df.loc[i, (col+'_72')] + df.loc[i-2, col]
        
    
        #caso in cui c'è un buco di un giorno
        if (df.loc[i, 'user.id'] == df.loc[i-1, 'user.id'] and 
            df.loc[i, 'time'] - timedelta(days=2) == df.loc[i-1, 'time']):
                                                                     
                df.loc[i, (col+'_72')] = df.loc[i, (col+'_72')] + df.loc[i-1, col]
        
    

In [21]:
# calcolo il numero di incontri nei giorni precedenti per tutti i tipi di relazione
relationships_names = ['BlogLivejournalTwitter', 'CloseFriend', 'FacebookAllTaggedPhotos','PoliticalDiscussant', 
                       'SocializeTwicePerWeek', 'Unknown']

for column_name in relationships_names:
    proximity_48_72(df_pro_rel_daily,column_name)
    
df_pro_rel_daily.head()



,user.id,time,BlogLivejournalTwitter,CloseFriend,FacebookAllTaggedPhotos,PoliticalDiscussant,SocializeTwicePerWeek,Unknown,BlogLivejournalTwitter_48,BlogLivejournalTwitter_72,CloseFriend_48,CloseFriend_72,FacebookAllTaggedPhotos_48,FacebookAllTaggedPhotos_72,PoliticalDiscussant_48,PoliticalDiscussant_72,SocializeTwicePerWeek_48,SocializeTwicePerWeek_72,Unknown_48,Unknown_72
0,1,2009-01-01,0,0,0,0,0,4,0,0,0,0,0,0,0,0,0,0,4,4
1,1,2009-01-02,0,0,0,0,0,28,0,0,0,0,0,0,0,0,0,0,32,32
2,1,2009-01-03,0,0,0,0,0,16,0,0,0,0,0,0,0,0,0,0,44,48
3,1,2009-01-04,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,17,45
4,1,2009-01-05,0,0,0,0,0,3,0,0,0,0,0,0,0,0,0,0,4,20


In [22]:
#rinomino le colonne delle 24 ore
df_pro_rel_daily.rename(columns={'BlogLivejournalTwitter':'BlogLivejournalTwitter_24', 'CloseFriend':'CloseFriend_24',
                     'FacebookAllTaggedPhotos':'FacebookAllTaggedPhotos_24','PoliticalDiscussant':'PoliticalDiscussant_24', 
                       'SocializeTwicePerWeek':'SocializeTwicePerWeek_24', 'Unknown':'Unknown_24'}, inplace=True)
df_pro_rel_daily.head()

,user.id,time,BlogLivejournalTwitter_24,CloseFriend_24,FacebookAllTaggedPhotos_24,PoliticalDiscussant_24,SocializeTwicePerWeek_24,Unknown_24,BlogLivejournalTwitter_48,BlogLivejournalTwitter_72,CloseFriend_48,CloseFriend_72,FacebookAllTaggedPhotos_48,FacebookAllTaggedPhotos_72,PoliticalDiscussant_48,PoliticalDiscussant_72,SocializeTwicePerWeek_48,SocializeTwicePerWeek_72,Unknown_48,Unknown_72
0,1,2009-01-01,0,0,0,0,0,4,0,0,0,0,0,0,0,0,0,0,4,4
1,1,2009-01-02,0,0,0,0,0,28,0,0,0,0,0,0,0,0,0,0,32,32
2,1,2009-01-03,0,0,0,0,0,16,0,0,0,0,0,0,0,0,0,0,44,48
3,1,2009-01-04,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,17,45
4,1,2009-01-05,0,0,0,0,0,3,0,0,0,0,0,0,0,0,0,0,4,20


In [23]:
#merge
#df_pro_rel_daily.rename(columns={'time': "Time"}, inplace=True)
df2 = df.merge(df_pro_rel_daily, how='left', left_on=['user_id', 'time'], right_on=['user.id', 'time'])
df2.drop(columns=['user.id'],inplace=True)
df2.head()

,user_id,time,cough,congestion,fever,nausea,depressed,stress,n_contacts_24,n_contacts_48,...,CloseFriend_48,CloseFriend_72,FacebookAllTaggedPhotos_48,FacebookAllTaggedPhotos_72,PoliticalDiscussant_48,PoliticalDiscussant_72,SocializeTwicePerWeek_48,SocializeTwicePerWeek_72,Unknown_48,Unknown_72
0,2,2009-01-09,0,0,0,0,1,1,68.0,110.0,...,1.0,1.0,27.0,27.0,31.0,32.0,19.0,19.0,74.0,182.0
1,2,2009-01-10,0,0,0,0,1,1,107.0,175.0,...,1.0,1.0,26.0,27.0,30.0,31.0,18.0,19.0,140.0,181.0
2,2,2009-01-11,0,0,0,0,0,0,95.0,202.0,...,14.0,15.0,27.0,53.0,15.0,45.0,2.0,20.0,175.0,208.0
3,2,2009-01-12,0,0,0,1,0,0,185.0,280.0,...,23.0,23.0,37.0,37.0,25.0,25.0,3.0,3.0,243.0,350.0
4,2,2009-01-13,0,0,0,0,0,0,22.0,207.0,...,11.0,25.0,13.0,40.0,13.0,28.0,2.0,4.0,194.0,262.0


In [24]:
# aggiungo a symptomps una variabile di controllo che mi dice chi ha almeno uno dei primi 4 sintomi
df_sym['Sick'] =  (df_sym[['cough','congestion','fever','nausea']].sum(axis=1) != 0)
df_sym.head()

,user_id,time,cough,congestion,fever,nausea,depressed,stress,Sick
0,2,2009-01-09,0,0,0,0,1,1,False
1,2,2009-01-10,0,0,0,0,1,1,False
2,2,2009-01-11,0,0,0,0,0,0,False
3,2,2009-01-12,0,0,0,1,0,0,True
4,2,2009-01-13,0,0,0,0,0,0,False


In [25]:
# merge di proximity e symptomps. per avere conteggio incontri con malati
df_pro_sick = df_pro.merge(df_sym, how='left', left_on=['remote.user','time'], right_on=['user_id', 'time'])

#df_pro_sick.drop(columns=['prob2','user_id','cough','congestion','fever','nausea', 'depressed','stress','time_contagion'],inplace=True)

df_pro_sick['Sick'].fillna(False, inplace=True)

df_pro_sick.head() # i NaN verranno eliminati dal succcessivo groupby

,user.id,remote.user,time,prob2,user_id,cough,congestion,fever,nausea,depressed,stress,Sick
0,46,20,2009-01-01,0.628,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
1,20,46,2009-01-01,0.628,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
2,46,30,2009-01-01,0.371,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
3,30,46,2009-01-01,0.371,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
4,61,61,2009-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False


In [26]:
# conteggio di contatti con persone infette (totale e per singoli sintomi)
df_pro_sick_daily = df_pro_sick.groupby(['user.id','time'], as_index=False).agg({'Sick':'sum',
                                                                                 'cough':'sum',
                                                                                 'congestion':'sum',
                                                                                 'fever':'sum',
                                                                                 'nausea':'sum', 
                                                                                 'depressed':'sum',
                                                                                 'stress':'sum'})
df_pro_sick_daily.rename(columns={'Sick': "sick_contacts",
                                  'cough':'cough_contacts',
                                  'congestion':'congestion_contacts',
                                  'fever':'fever_contacts',
                                  'nausea':'nausea_contacts', 
                                  'depressed':'depressed_contacts',
                                  'stress':'stress_contacts'}, inplace=True)
df_pro_sick_daily.head(20)

,user.id,time,sick_contacts,cough_contacts,congestion_contacts,fever_contacts,nausea_contacts,depressed_contacts,stress_contacts
0,1,2009-01-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,2009-01-02,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,2009-01-03,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1,2009-01-04,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1,2009-01-05,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,1,2009-01-06,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,1,2009-01-08,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,1,2009-01-09,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,1,2009-01-12,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,1,2009-01-13,6.0,4.0,6.0,0.0,0.0,0.0,0.0


In [27]:
# calcolo il numero di contatti nei giorni precedenti per tutti i sintomi
symptoms_names = ['cough_contacts','congestion_contacts','fever_contacts','nausea_contacts', 
                  'depressed_contacts','stress_contacts','sick_contacts']

for column_name in symptoms_names:
    proximity_48_72(df_pro_sick_daily,column_name)
    
df_pro_sick_daily.head(20)

,user.id,time,sick_contacts,cough_contacts,congestion_contacts,fever_contacts,nausea_contacts,depressed_contacts,stress_contacts,cough_contacts_48,...,fever_contacts_48,fever_contacts_72,nausea_contacts_48,nausea_contacts_72,depressed_contacts_48,depressed_contacts_72,stress_contacts_48,stress_contacts_72,sick_contacts_48,sick_contacts_72
0,1,2009-01-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,2009-01-02,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,2009-01-03,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,2009-01-04,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,2009-01-05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,...,0,0,0,0,0,0,0,0,0,0
5,1,2009-01-06,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,...,0,0,0,0,0,0,0,0,0,0
6,1,2009-01-08,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,...,0,0,0,0,0,0,0,0,0,0
7,1,2009-01-09,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,...,0,0,0,0,0,0,0,0,0,0
8,1,2009-01-12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,...,0,0,0,0,0,0,0,0,0,0
9,1,2009-01-13,6.0,4.0,6.0,0.0,0.0,0.0,0.0,4,...,0,0,0,0,0,0,0,0,6,6


In [28]:
#rinomino le colonne delle 24 ore
df_pro_sick_daily.rename(columns={'sick_contacts': "sick_contacts_24",
                                  'cough_contacts':'cough_contacts_24',
                                  'congestion_contacts':'congestion_contacts_24',
                                  'fever_contacts':'fever_contacts_24',
                                  'nausea_contacts':'nausea_contacts_24', 
                                  'depressed_contacts':'depressed_contacts_24',
                                  'stress_contacts':'stress_contacts_24'}, inplace=True)
df_pro_sick_daily.head(20)

,user.id,time,sick_contacts_24,cough_contacts_24,congestion_contacts_24,fever_contacts_24,nausea_contacts_24,depressed_contacts_24,stress_contacts_24,cough_contacts_48,...,fever_contacts_48,fever_contacts_72,nausea_contacts_48,nausea_contacts_72,depressed_contacts_48,depressed_contacts_72,stress_contacts_48,stress_contacts_72,sick_contacts_48,sick_contacts_72
0,1,2009-01-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,2009-01-02,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,2009-01-03,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,2009-01-04,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,2009-01-05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,...,0,0,0,0,0,0,0,0,0,0
5,1,2009-01-06,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,...,0,0,0,0,0,0,0,0,0,0
6,1,2009-01-08,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,...,0,0,0,0,0,0,0,0,0,0
7,1,2009-01-09,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,...,0,0,0,0,0,0,0,0,0,0
8,1,2009-01-12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,...,0,0,0,0,0,0,0,0,0,0
9,1,2009-01-13,6.0,4.0,6.0,0.0,0.0,0.0,0.0,4,...,0,0,0,0,0,0,0,0,6,6


In [29]:
#merge
#df_pro_sick_daily.rename(columns={'time': "Time"}, inplace=True)
df2 = df2.merge(df_pro_sick_daily, how='left', left_on=['user_id', 'time'], right_on=['user.id', 'time'])
df2.drop(columns=['user.id'],inplace=True)
df2.head()

,user_id,time,cough,congestion,fever,nausea,depressed,stress,n_contacts_24,n_contacts_48,...,fever_contacts_48,fever_contacts_72,nausea_contacts_48,nausea_contacts_72,depressed_contacts_48,depressed_contacts_72,stress_contacts_48,stress_contacts_72,sick_contacts_48,sick_contacts_72
0,2,2009-01-09,0,0,0,0,1,1,68.0,110.0,...,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0
1,2,2009-01-10,0,0,0,0,1,1,107.0,175.0,...,0.0,0,0.0,0,0.0,0,0.0,0,96.0,96
2,2,2009-01-11,0,0,0,0,0,0,95.0,202.0,...,0.0,0,0.0,0,0.0,0,0.0,0,98.0,98
3,2,2009-01-12,0,0,0,1,0,0,185.0,280.0,...,0.0,0,0.0,0,0.0,0,0.0,0,68.0,164
4,2,2009-01-13,0,0,0,0,0,0,22.0,207.0,...,0.0,0,0.0,0,0.0,0,0.0,0,74.0,76


In [30]:
# creo una possibile target variable (non tengo conto di stress e depressed)
df2['Target'] =  df2[['cough','congestion','fever','nausea']].sum(axis=1)
df2['Target'] = df2['Target'].apply(str)

# target = 0 sano
# target = 0.5 1 sintomo
# target = 0.66 2 sintomi
# target = 0.83 3 sintomi
# target = 1 4 sintomi
df2['Target_prob'] = df2['Target'].replace({'0': 0, '1': 0.5, '2': 0.66, '3': 0.83, '4': 1})

# target true/false
df2['Target_bin'] =  (df2[['cough','congestion','fever','nausea']].sum(axis=1) != 0)
df2.head(20)

,user_id,time,cough,congestion,fever,nausea,depressed,stress,n_contacts_24,n_contacts_48,...,nausea_contacts_72,depressed_contacts_48,depressed_contacts_72,stress_contacts_48,stress_contacts_72,sick_contacts_48,sick_contacts_72,Target,Target_prob,Target_bin
0,2,2009-01-09,0,0,0,0,1,1,68.0,110.0,...,0,0.0,0,0.0,0,0.0,0,0,0.0,False
1,2,2009-01-10,0,0,0,0,1,1,107.0,175.0,...,0,0.0,0,0.0,0,96.0,96,0,0.0,False
2,2,2009-01-11,0,0,0,0,0,0,95.0,202.0,...,0,0.0,0,0.0,0,98.0,98,0,0.0,False
3,2,2009-01-12,0,0,0,1,0,0,185.0,280.0,...,0,0.0,0,0.0,0,68.0,164,1,0.5,True
4,2,2009-01-13,0,0,0,0,0,0,22.0,207.0,...,0,0.0,0,0.0,0,74.0,76,0,0.0,False
5,2,2009-01-14,0,0,0,0,1,0,57.0,79.0,...,0,0.0,0,0.0,0,63.0,129,0,0.0,False
6,2,2009-01-15,0,0,0,0,0,0,108.0,165.0,...,0,0.0,0,1.0,1,111.0,119,0,0.0,False
7,2,2009-01-16,0,0,0,0,1,0,18.0,126.0,...,0,0.0,0,1.0,1,60.0,115,0,0.0,False
8,2,2009-01-16,0,0,0,0,1,0,18.0,126.0,...,0,0.0,0,1.0,1,60.0,115,0,0.0,False
9,2,2009-01-17,0,0,0,0,0,0,3.0,21.0,...,0,0.0,0,0.0,1,4.0,60,0,0.0,False


In [31]:
df2 = df2.sort_values(by='Target')

In [32]:
df2 = df2.drop_duplicates(subset=['user_id', 'time'], keep="last") #tolgo casi ridondanti (risposte date più volte in un giorno)

In [33]:
df2 = df2.sort_values(['user_id','time'])
df2.head()

,user_id,time,cough,congestion,fever,nausea,depressed,stress,n_contacts_24,n_contacts_48,...,nausea_contacts_72,depressed_contacts_48,depressed_contacts_72,stress_contacts_48,stress_contacts_72,sick_contacts_48,sick_contacts_72,Target,Target_prob,Target_bin
0,2,2009-01-09,0,0,0,0,1,1,68.0,110.0,...,0,0.0,0,0.0,0,0.0,0,0,0.0,False
1,2,2009-01-10,0,0,0,0,1,1,107.0,175.0,...,0,0.0,0,0.0,0,96.0,96,0,0.0,False
2,2,2009-01-11,0,0,0,0,0,0,95.0,202.0,...,0,0.0,0,0.0,0,98.0,98,0,0.0,False
3,2,2009-01-12,0,0,0,1,0,0,185.0,280.0,...,0,0.0,0,0.0,0,68.0,164,1,0.5,True
4,2,2009-01-13,0,0,0,0,0,0,22.0,207.0,...,0,0.0,0,0.0,0,74.0,76,0,0.0,False


In [34]:
df2.to_csv('df_aggiornato.csv', encoding='utf-8')

In [35]:
df2.describe()

,user_id,cough,congestion,fever,nausea,depressed,stress,n_contacts_24,n_contacts_48,n_contacts_72,...,depressed_contacts_24,stress_contacts_24,cough_contacts_48,congestion_contacts_48,fever_contacts_48,nausea_contacts_48,depressed_contacts_48,stress_contacts_48,sick_contacts_48,Target_prob
count,3071.000000,3071.000000,3071.000000,3071.000000,3071.000000,3071.000000,3071.000000,2567.000000,2567.000000,2567.000000,...,2567.000000,2567.000000,2567.00000,2567.000000,2567.000000,2567.000000,2567.000000,2567.000000,2567.000000,3071.000000
mean,43.009118,0.145555,0.216867,0.009443,0.031911,0.074243,0.167047,471.089209,915.747955,1341.802493,...,26.077133,70.801325,118.85859,137.248929,5.781457,16.443709,51.444877,137.645111,185.296845,0.160287
std,23.242748,0.352717,0.412179,0.096732,0.175793,0.262208,0.373078,525.266422,931.542895,1307.187387,...,70.953062,140.385366,228.21055,237.687154,40.566797,54.981764,109.314644,227.010048,280.230419,0.261549
min,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,...,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,23.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,104.000000,237.000000,385.500000,...,0.000000,0.000000,1.00000,5.000000,0.000000,0.000000,0.000000,2.000000,11.000000,0.000000
50%,46.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,313.000000,668.000000,989.000000,...,0.000000,11.000000,26.00000,47.000000,0.000000,0.000000,3.000000,40.000000,81.000000,0.000000
75%,62.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,674.000000,1344.000000,1946.500000,...,12.000000,86.000000,139.00000,175.000000,0.000000,4.000000,46.000000,186.500000,248.000000,0.500000
max,80.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,6927.000000,13279.000000,13557.000000,...,1198.000000,1884.000000,3244.00000,3274.000000,688.000000,825.000000,1217.000000,3318.000000,3278.000000,1.000000
